### Bulding Geneformer-Like Dataset From Yeast Master Matrix

#### Building .loom file

##### Replacing NaNs by 0s

In [ ]:
import pandas as pd

file_path = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/original_yeast_master_matrix_sgd_copy.csv"

# Load CSV
df = pd.read_csv(file_path, sep='\t', index_col=0) # Important to keep sep='\t', since that's how it was saved 

# print(df.shape)

# Replace NaNs with 0
df.fillna(0, inplace=True)

# Save the cleaned CSV
df.to_csv(file_path, sep='\t')

print("NaNs replaced with 0")

##### Translating to .loom

In [ ]:
import pandas as pd
import numpy as np
import loompy

input_file_path = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/original_yeast_master_matrix_sgd_copy.csv"
output_file_path = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_master_matrix_sgd.loom"

# Load CSV
df = pd.read_csv(input_file_path, sep='\t', index_col=0) # Important to keep sep='\t', since that's how it was saved 

# Compute total read counts for each experiment (equivalent to cell in geneformer)
n_counts = df.sum(axis=0).astype(np.float32)  # Sum across genes for each column 

# Prepare row attributes (Gene IDs → Ensembl IDs assumed to be index)
row_attrs = {"ensembl_id": df.index.tolist()}  # Ensure index has Ensembl IDs

# Prepare column attributes (Cells & their total read counts)
col_attrs = {
    # "exp_name": df.columns.tolist(),  # Experiment names
    "n_counts": n_counts.values,   # Total counts per column (experiment)
}

# Convert DataFrame to Loom format & save
loompy.create(output_file_path, df.values.astype(np.float32), row_attrs, col_attrs)

print(f"Loom file saved as: {output_file_path}")

In [ ]:
import numpy as np
print(np.__version__)  # Should print something like 1.26.4

# Had to downgrade numpy < 2.0

##### Verifying .loom file

In [ ]:
import loompy

input_loom_file_path = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_master_matrix_sgd.loom"

with loompy.connect(input_loom_file_path) as ds:
    # Print general metadata
    print("Row attributes:", ds.ra.keys())  # Should contain 'ensembl_id'
    print("Column attributes:", ds.ca.keys())  # Should contain 'n_counts'
    print("Data shape (genes x exp columns):", ds.shape)

    # Print first 5 genes (rows) and their attributes
    print("\nFirst 5 Row Attributes:")
    for key in ds.ra.keys():
        print(f"{key}: {ds.ra[key][:5]}")  # Print first 5 values of each row attribute

    # print(f"exp_name: {ds.ca['exp_name'][:1]}")
    print(f"n_counts: {ds.ca['n_counts'][:5]}")

#### Generating Dictionaries 

##### Tokens Dictionary

###### Load example to see the intuition

In [ ]:
import pickle

# Path to the token dictionary file
token_dict_file = "/home/logs/jtorresb/Geneformer/geneformer/token_dictionary_gc95M.pkl"

# Function to inspect the token dictionary
def inspect_token_dictionary(file_path, num_samples=10):
    with open(file_path, "rb") as f:
        token_dict = pickle.load(f)
    
    print(f"Token dictionary type: {type(token_dict)}")
    print(f"Total tokens: {len(token_dict)}")
    print("First 10 token entries:")
    sample_items = list(token_dict.items())[:num_samples]
    for key, value in sample_items:
        print(f"{key}: {value}")

# Run the inspection
inspect_token_dictionary(token_dict_file)

###### Generating Token Dictionary

In [ ]:
import pandas as pd
import pickle

# File paths
csv_file = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/original_yeast_master_matrix_sgd.csv"
output_pkl = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_token_dict.pkl"

# Load CSV (Ensure YORFs are the index)
df = pd.read_csv(csv_file, sep='\t', index_col=0)

# Extract yeast ORFs (YORFs) from index
yorfs = df.index.tolist()

# Optional: Sort alphabetically for consistency
yorfs.sort()

# Initialize token dictionary with special tokens
token_dict = {
    "<pad>": 0,
    "<mask>": 1,
    "<cls>": 2,
    "<eos>": 3,
}

# Assign unique token IDs starting from 4
for i, gene_id in enumerate(yorfs, start=4):
    token_dict[gene_id] = i

# Save dictionary as a pickle file
with open(output_pkl, "wb") as f:
    pickle.dump(token_dict, f)

print(f"Token dictionary saved as: {output_pkl}")

##### Medians Dictionary

###### Inspecting example first

In [ ]:
import pickle

# Path to the median dictionary file
median_dict_file = "/home/logs/jtorresb/Geneformer/geneformer/gene_median_dictionary_gc95M.pkl"

# Function to inspect the median dictionary
def inspect_median_dictionary(file_path, num_samples=10):
    with open(file_path, "rb") as f:
        median_dict = pickle.load(f)
    
    print(f"Median dictionary type: {type(median_dict)}")
    print(f"Total genes in dictionary: {len(median_dict)}")
    print("First 10 median entries:")
    sample_items = list(median_dict.items())[:num_samples]
    for key, value in sample_items:
        print(f"{key}: {value}")

# Run the inspection
inspect_median_dictionary(median_dict_file)

In [ ]:
import pandas as pd
import numpy as np
import pickle

# File paths
csv_file = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/original_yeast_master_matrix_sgd_copy.csv" # Copy already replaced NaNs by 0s
output_pkl = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_median_dict.pkl"

# Load CSV (genes as index, experiments as columns)
df = pd.read_csv(csv_file, sep='\t', index_col=0)

# Compute nonzero medians for each gene
median_dict = {}
for gene in df.index:
    nonzero_values = df.loc[gene][df.loc[gene] != 0]  # Ignore zeros
    if not nonzero_values.empty:
        median_dict[gene] = np.median(nonzero_values)  # Compute median
    else:
        median_dict[gene] = 0  # If all values are zero, set median to 0

# Save dictionary as a pickle file
with open(output_pkl, "wb") as f:
    pickle.dump(median_dict, f)

print(f"Median dictionary saved as: {output_pkl}")


#### Tokenizing Loom File

##### Quick model_input_size ChecK

In [ ]:
import loompy
import pickle
import numpy as np

# File paths (update these as needed)
loom_file_path = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_master_matrix_sgd.loom"
median_dict_file = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_median_dict.pkl"
token_dict_file = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_token_dict.pkl"

# Load the median dictionary and token dictionary
with open(median_dict_file, "rb") as f:
    median_dict = pickle.load(f)
with open(token_dict_file, "rb") as f:
    token_dict = pickle.load(f)

# This function simulates tokenization for one cell:
# It normalizes each gene's expression by its nonzero median,
# then ranks the genes by the normalized expression (highest first),
# and finally maps the gene IDs to token IDs using token_dict.
def tokenize_cell(expr_vector, ensembl_ids):
    gene_norms = []
    for idx, expr in enumerate(expr_vector):
        # Only consider genes with nonzero expression
        if expr != 0:
            gene_id = ensembl_ids[idx]
            # Use the median if available and > 0; skip otherwise
            if gene_id in median_dict and median_dict[gene_id] > 0:
                norm_val = expr / median_dict[gene_id]
                gene_norms.append((gene_id, norm_val))
    # Sort genes by normalized value in descending order
    gene_norms_sorted = sorted(gene_norms, key=lambda x: -x[1])
    token_seq = []
    for gene_id, _ in gene_norms_sorted:
        # Only add tokens for genes that exist in the token dictionary.
        # (If a gene is missing, it will be skipped.)
        if gene_id in token_dict:
            token_seq.append(token_dict[gene_id])
    return token_seq

# Open the loom file and compute tokenized sequence lengths for a subset of cells.
token_lengths = []

with loompy.connect(loom_file_path) as ds:
    ensembl_ids = ds.ra["ensembl_id"]  # Array of gene IDs (rows)
    num_genes, num_cells = ds.shape
    print(f"Loom file shape (genes x cells): {ds.shape}")
    
    # Process a subset of cells (e.g., first 100 cells)
    num_cells_to_process = min(num_cells, 100)
    
    for cell_idx in range(num_cells_to_process):
        # Get the expression vector for the cell (all genes)
        expr_vector = ds[:, cell_idx].astype(np.float32)
        # Tokenize the cell's gene expression
        token_seq = tokenize_cell(expr_vector, ensembl_ids)
        token_lengths.append(len(token_seq))
    
# Print statistics about the tokenized sequence lengths
token_lengths = np.array(token_lengths)
print(f"Processed {num_cells_to_process} cells.")
print("Token sequence lengths (number of tokens per cell):")
print(token_lengths)
print(f"Average token sequence length: {np.mean(token_lengths):.1f}")
print(f"Median token sequence length: {np.median(token_lengths):.1f}")


##### Tokenization using TranscriptomeTokenizer

In [ ]:
from geneformer import TranscriptomeTokenizer

tk = TranscriptomeTokenizer(custom_attr_name_dict=None, nproc=2, chunk_size=512, model_input_size=4096, 
                            special_token=True, collapse_gene_ids=True, gene_median_file='/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_median_dict.pkl',
                            token_dictionary_file='/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_token_dict.pkl', gene_mapping_file=None)
# special_token = False was giving problems with "ensemble_ids_collapse"

In [ ]:
tk.tokenize_data(data_directory="/home/logs/jtorresb/Geneformer/yeast/yeast_data/output",
                 output_directory="/home/logs/jtorresb/Geneformer/yeast/yeast_data/output",
                 output_prefix="yeast_master_matrix_sgd", 
                 file_format="loom",
                 use_generator=False)

##### Checking .dataset File

In [ ]:
from datasets import load_from_disk

# Replace the path with the location of your .dataset file
dataset_path = "/home/logs/jtorresb/Geneformer/yeast/yeast_data/output/yeast_master_matrix_sgd.dataset"
data = load_from_disk(dataset_path)

# Convert the dataset to a pandas DataFrame
df = data.to_pandas()

# Display the first few rows of the DataFrame
print(df.head(10))


# Filter rows where the length is not 4096
not_4096 = df[df["length"] != 4096]
print(f"Total rows: {len(not_4096)}") # All rows should have length 4096 (interesting to comment)

                                           input_ids  length
0  [2, 2821, 1289, 612, 3670, 87, 2976, 4704, 590...    4096
1  [2, 4580, 6708, 5639, 2013, 6383, 4025, 2786, ...    4096
2  [2, 6708, 4580, 2132, 2013, 4462, 5639, 2786, ...    4096
3  [2, 6708, 4580, 3453, 2013, 2487, 5639, 2786, ...    4096
4  [2, 4580, 6708, 1289, 2786, 2487, 5300, 2013, ...    4096
5  [2, 4580, 1289, 2821, 4462, 5901, 1031, 5323, ...    4096
6  [2, 4580, 1289, 2821, 4462, 1031, 5901, 5323, ...    4096
7  [2, 4580, 1289, 2132, 2821, 5901, 5323, 1031, ...    4096
8  [2, 4580, 1289, 2821, 2132, 5323, 1031, 5901, ...    4096
9  [2, 4580, 1289, 2132, 2821, 1031, 5323, 5941, ...    4096
Total rows: 1550


In [15]:
# Replace the path with the location of your .dataset file
dataset_path = "/home/logs/jtorresb/Geneformer/Genecorpus/example_input_files/gene_classification/dosage_sensitive_tfs/gc-30M_sample50k.dataset"
data = load_from_disk(dataset_path)

# Convert the dataset to a pandas DataFrame
df = data.to_pandas()

# Display the first few rows of the DataFrame
print(df.head(10))


                                           input_ids  length
0  [6751, 11578, 5918, 4500, 5707, 16979, 6705, 1...     886
1  [3911, 14241, 7375, 6385, 7020, 15538, 1580, 1...      88
2  [20154, 9034, 3623, 10248, 6560, 10532, 6171, ...      17
3  [20515, 16829, 9401, 10221, 17269, 5205, 17295...      25
4  [2183, 5592, 8163, 5074, 14116, 5972, 3584, 19...    1330
5  [4260, 9049, 3854, 5262, 9843, 17033, 17084, 1...     543
6  [17454, 482, 8115, 20854, 8675, 6297, 20131, 1...      88
7  [10158, 11924, 24647, 861, 4733, 3471, 4091, 1...    1021
8  [5332, 20154, 6380, 2326, 10219, 3882, 13237, ...      55
9  [10390, 16322, 15588, 8874, 5716, 1667, 2565, ...      70
